In [1]:
N_JOBS = 6
N_MACHINES = 4

In [2]:
import random

def generateData(seed=0):
    PROC = [ [ 0 for m in range(N_MACHINES) ] for j in range(N_JOBS) ]
    random.seed(seed)
    for j in range(N_JOBS):
        for m in range(N_MACHINES):
            PROC[j][m] = random.randint(1,9)
    return PROC

In [3]:
PROC = generateData(8548)

In [4]:
PROC

[[4, 1, 6, 2],
 [1, 4, 4, 4],
 [1, 1, 2, 2],
 [1, 6, 8, 8],
 [9, 8, 7, 3],
 [7, 4, 3, 7]]

In [5]:
import pulp

In [6]:
def val(x):
    return int(pulp.value(x))

In [7]:
prob = pulp.LpProblem("JobScheduling",pulp.LpMinimize)

`J[j]` defines the sequencing of jobs 'j' and is just a permutation of the job number:

In [8]:
JJ = pulp.LpVariable.dicts("J", (range(N_JOBS), range(N_JOBS)),
                           lowBound=0, upBound=1, cat='Integer')

In [9]:
def job(n):
    for j in range(N_JOBS):
        if val(JJ[n][j])==1:
            return j

In [10]:
def jobSequence():
    acc = []
    for j in range(N_JOBS):
        for jj in range(N_JOBS):
            if pulp.value(JJ[j][jj])==1:
                acc.append(jj)
    return acc

In [11]:
def proc(job, machine):
    global PROC
    return pulp.lpSum( [ PROC[j][machine] * JJ[j][job] for j in range(N_JOBS) ] ) 

`WAIT[j][m]` describes the wait time of job `j` before machine `m`

In [12]:
Wait = pulp.LpVariable.dicts("WAIT", (range(N_JOBS), range(N_MACHINES)),
                          lowBound=0, cat='Integer')

`IDLE[j][m]` describes the idle time of machine `m` before processing job `j` 

In [13]:
Idle = pulp.LpVariable.dicts("IDLE", (range(N_JOBS), range(N_MACHINES)),
                             lowBound=0, cat='Integer')

`START[j][m]` describes the start time of machine `m` processing job `j`

In [14]:
Start = pulp.LpVariable.dicts("START", (range(N_JOBS), range(N_MACHINES)),
                          lowBound=0, cat='Integer')

`STOP[j][m]` describes the stop time of machine `m` after processing job `j`

In [15]:
Stop = pulp.LpVariable.dicts("STOP", (range(N_JOBS), range(N_MACHINES)),
                          lowBound=0, cat='Integer')

In [16]:
prob += Stop[N_JOBS-1][N_MACHINES-1]

In [17]:
for j in range(N_JOBS):
    prob += pulp.lpSum( [ JJ[j][jj] for jj in range(N_JOBS) ] ) == 1
    prob += pulp.lpSum( [ JJ[jj][j] for jj in range(N_JOBS) ] ) == 1

In [18]:
for m in range(N_MACHINES):
    for j in range(N_JOBS):
        prob += pulp.lpSum( [ Idle[ji][m] + proc(ji, m) for ji in range(j) ] ) + Idle[j][m] == Start[j][m]

In [19]:
for m in range(N_MACHINES):
    for j in range(N_JOBS):
        prob += pulp.lpSum( [ Wait[j][mi] + proc(j, mi) for mi in range(m) ] ) + Wait[j][m] == Start[j][m]

In [20]:
for j in range(N_JOBS):
    for m in range(N_MACHINES):
        prob += Start[j][m] + proc(j,m) == Stop[j][m]

In [21]:
solvers = pulp.listSolvers(onlyAvailable=True) 
solver = pulp.getSolver(solvers[0], msg=0)
prob.solve(solver)

1

In [22]:
def schedule():
    row = ""
    row += '|---------|'
    for m in range(N_MACHINES):
        row += f"------------|" \
                f"------------|"
    row += '\n'
    row += '|         |'
    for m in range(N_MACHINES):
        row += f"            |" \
                f" Machine: {m:1d} |"
    row += '\n'
    for j in range(N_JOBS):
        row += '|---------|'
        for m in range(N_MACHINES):
            row += f"------------|" \
                   f"------------|"
        row += '\n'
        row += '|         |'
        for m in range(N_MACHINES):
            row += f"       {' ':4s} | " \
                   f"Idle: {val(Idle[j][m]):4d} |"
        row += '\n'
        row += '|---------|'
        for m in range(N_MACHINES):
            row += f"------------|" \
                   f"------------|"
        row += '\n'
        row += '|         |'
        for m in range(N_MACHINES):
            row += f"       {' ':4s} | " \
                   f"Start:{val(Start[j][m]):4d} |"  
        row += '\n' 
        row += f'| Job: {job(j):2d} |'
        for m in range(N_MACHINES):
            row += f" Wait: {val(Wait[j][m]):4d} | " \
                   f"Proc: {val(proc(j,m)):4d} |"
        row += '\n'
        row += '|         |'
        for m in range(N_MACHINES):
            row += f"       {' ':4s} | " \
                   f"Stop: {val(Stop[j][m]):4d} |"  
        row += '\n' 
    row += '|---------|'
    for m in range(N_MACHINES):
        row += f"------------|" \
                f"------------|"
    row += '\n'
    return row

The processing times per job and machine:

In [23]:
PROC

[[4, 1, 6, 2],
 [1, 4, 4, 4],
 [1, 1, 2, 2],
 [1, 6, 8, 8],
 [9, 8, 7, 3],
 [7, 4, 3, 7]]

The optimal job schedule:

In [24]:
print(jobSequence())

[5, 2, 0, 1, 3, 4]


The processing time of the optimal job schedule:

In [25]:
int(pulp.value(prob.objective))

40

The optimal job schedule in detail:

In [26]:
print(schedule())

|---------|------------|------------|------------|------------|------------|------------|------------|------------|
|         |            | Machine: 0 |            | Machine: 1 |            | Machine: 2 |            | Machine: 3 |
|---------|------------|------------|------------|------------|------------|------------|------------|------------|
|         |            | Idle:    0 |            | Idle:    1 |            | Idle:    6 |            | Idle:    8 |
|---------|------------|------------|------------|------------|------------|------------|------------|------------|
|         |            | Start:   0 |            | Start:   1 |            | Start:   6 |            | Start:   8 |
| Job:  5 | Wait:    0 | Proc:    1 | Wait:    0 | Proc:    1 | Wait:    4 | Proc:    2 | Wait:    0 | Proc:    2 |
|         |            | Stop:    1 |            | Stop:    2 |            | Stop:    8 |            | Stop:   10 |
|---------|------------|------------|------------|------------|---------

## USING GREEDY ALGORITHM ##

In [27]:
def greedy_schedule(PROC):
    job_assignments = [-1] * N_JOBS  # Initialize job assignments
    
    # arrange the jobs by the sum of processing times
    sorted_jobs = sorted(range(N_JOBS), key=lambda j: sum(PROC[j]))
    
    # greedily alocate jobs to machines
    for j in sorted_jobs:
        min_machine = min(range(N_MACHINES), key=lambda m: sum(PROC[i][m] for i in range(N_JOBS) if job_assignments[i] == m))
        job_assignments[j] = min_machine
        
    # Calculating the start, wait, and stop times for each job on each machine
    start_times = [[0] * N_MACHINES for _ in range(N_JOBS)]
    wait_times = [[0] * N_MACHINES for _ in range(N_JOBS)]
    stop_times = [[0] * N_MACHINES for _ in range(N_JOBS)]
    
    for j in range(N_JOBS):
        for m in range(N_MACHINES):
            if job_assignments[j] == m:
                if j == 0:
                    start_times[j][m] = 0
                else:
                    start_times[j][m] = stop_times[j - 1][m]
                wait_times[j][m] = max(0, start_times[j][m] - stop_times[j - 1][m])
                stop_times[j][m] = start_times[j][m] + PROC[j][m]
            else:
                if j == 0:
                    start_times[j][m] = PROC[j][m]
                else:
                    start_times[j][m] = max(stop_times[j - 1][m], stop_times[j][job_assignments[j]])
                wait_times[j][m] = max(0, start_times[j][m] - stop_times[j - 1][m])
                stop_times[j][m] = start_times[j][m] + PROC[j][m]
    
    # Calculate total processing time (makespan)
    makespan = max(sum(PROC[j][m] for j in range(N_JOBS)) for m in range(N_MACHINES))
    
    return job_assignments, makespan




In [28]:
# to print the greedy algorithm result and best time
greedy_result, best_time = greedy_schedule(PROC)
print("Greedy algorithm result:", greedy_result)
print("Best time (makespan):", best_time)

Greedy algorithm result: [1, 2, 0, 0, 1, 3]
Best time (makespan): 30


In [29]:
def print_greedy_schedule(PROC, job_assignments):
    N_JOBS = len(PROC)
    N_MACHINES = len(PROC[0])

    # start, wait, and stop times for each job on each machine are Calculated
    start_times = [[0] * N_MACHINES for _ in range(N_JOBS)]
    wait_times = [[0] * N_MACHINES for _ in range(N_JOBS)]
    stop_times = [[0] * N_MACHINES for _ in range(N_JOBS)]
    
    for j in range(N_JOBS):
        for m in range(N_MACHINES):
            if job_assignments[j] == m:
                if j == 0:
                    start_times[j][m] = 0
                else:
                    start_times[j][m] = stop_times[j - 1][m]
                wait_times[j][m] = max(0, start_times[j][m] - stop_times[j - 1][m])
                stop_times[j][m] = start_times[j][m] + PROC[j][m]
            else:
                if j == 0:
                    start_times[j][m] = PROC[j][m]
                else:
                    start_times[j][m] = max(stop_times[j - 1][m], stop_times[j][job_assignments[j]])
                wait_times[j][m] = max(0, start_times[j][m] - stop_times[j - 1][m])
                stop_times[j][m] = start_times[j][m] + PROC[j][m]

     # To generate the schedule
    row = ""
    row += '|---------|'
    for m in range(N_MACHINES):
        row += f"------------|" \
               f"------------|"
    row += '\n'
    row += '|         |'
    for m in range(N_MACHINES):
        row += f"            |" \
               f" Machine: {m:1d} |"
    row += '\n'
    for j in range(N_JOBS):
        row += '|---------|'
        for m in range(N_MACHINES):
            row += f"------------|" \
                   f"------------|"
        row += '\n'
        row += '|         |'
        for m in range(N_MACHINES):
            row += f"       {' ':4s} | " \
                   f"Idle: {wait_times[j][m]:4d} |"
        row += '\n'
        row += '|---------|'
        for m in range(N_MACHINES):
            row += f"------------|" \
                   f"------------|"
        row += '\n'
        row += '|         |'
        for m in range(N_MACHINES):
            row += f"       {' ':4s} | " \
                   f"Start:{start_times[j][m]:4d} |"  
        row += '\n' 
        row += f'| Job: {j:2d} |'
        for m in range(N_MACHINES):
            row += f" Wait: {wait_times[j][m]:4d} | " \
                   f"Proc: {PROC[j][m]:4d} |"
        row += '\n'
        row += '|         |'
        for m in range(N_MACHINES):
            row += f"       {' ':4s} | " \
                   f"Stop: {stop_times[j][m]:4d} |"  
        row += '\n' 
    row += '|---------|'
    for m in range(N_MACHINES):
        row += f"------------|" \
               f"------------|"
    row += '\n'
    
    print(row)



In [30]:
# printing the schedule
print("\nSchedule:")
print_greedy_schedule(PROC, greedy_result)


Schedule:
|---------|------------|------------|------------|------------|------------|------------|------------|------------|
|         |            | Machine: 0 |            | Machine: 1 |            | Machine: 2 |            | Machine: 3 |
|---------|------------|------------|------------|------------|------------|------------|------------|------------|
|         |            | Idle:    4 |            | Idle:    0 |            | Idle:    6 |            | Idle:    2 |
|---------|------------|------------|------------|------------|------------|------------|------------|------------|
|         |            | Start:   4 |            | Start:   0 |            | Start:   6 |            | Start:   2 |
| Job:  0 | Wait:    4 | Proc:    4 | Wait:    0 | Proc:    1 | Wait:    6 | Proc:    6 | Wait:    2 | Proc:    2 |
|         |            | Stop:    8 |            | Stop:    1 |            | Stop:   12 |            | Stop:    4 |
|---------|------------|------------|------------|-----------

In [31]:
 #Calculate Accuracy
accuracy = (best_time / int(pulp.value(prob.objective))) * 100

In [32]:
# Print Results
print("Makespan (Best time) using greedy algorithm:", best_time)
print("Optimal Makespan from ILP Solution:", int(pulp.value(prob.objective)))
print("Accuracy of greedy algorithm solution:", accuracy, "%")

Makespan (Best time) using greedy algorithm: 30
Optimal Makespan from ILP Solution: 40
Accuracy of greedy algorithm solution: 75.0 %


## USING STOCHASTIC OPTIMISATION (Simulated Annealing) ##

In [33]:
import math

In [34]:
# begin simulated Annealing
def get_initial_solution():
    return [random.sample(range(N_MACHINES), N_MACHINES) for _ in range(N_JOBS)]


In [35]:
# deifine the objective function
def objective_function(schedule):
    makespan = max(sum(PROC[j][m] for m in schedule[j]) for j in range(N_JOBS))
    return makespan


In [36]:
def neighbor(solution):
    new_solution = solution[:]
    job_index = random.randint(0, N_JOBS - 1)
    machine_index1, machine_index2 = random.sample(range(N_MACHINES), 2)
    new_solution[job_index][machine_index1], new_solution[job_index][machine_index2] = new_solution[job_index][machine_index2], new_solution[job_index][machine_index1]
    return new_solution


In [37]:
def acceptance_probability(old_time, new_time, temperature):
    if new_time < old_time:
        return 1.0
    else:
        return math.exp((old_time - new_time) / temperature)


In [38]:

def simulated_annealing(max_iterations, initial_temperature, cooling_rate):
    current_solution = get_initial_solution()
    current_time = objective_function(current_solution)
    best_solution = current_solution
    best_time = current_time
    temperature = initial_temperature
    
    idle_times = [[0 for _ in range(N_MACHINES)] for _ in range(N_JOBS)]
    start_times = [[0 for _ in range(N_MACHINES)] for _ in range(N_JOBS)]
    stop_times = [[0 for _ in range(N_MACHINES)] for _ in range(N_JOBS)]
    
    for _ in range(max_iterations):
        neighbor_solution = neighbor(current_solution)
        neighbor_time = objective_function(neighbor_solution)
        if acceptance_probability(current_time, neighbor_time, temperature) > random.random():
            current_solution = neighbor_solution
            current_time = neighbor_time
            if current_time < best_time:
                best_solution = current_solution
                best_time = current_time
                
        # To Calculate idle, start, and stop times
        total_time = 0
        for j in range(N_JOBS):
            for m in range(N_MACHINES):
                total_time += PROC[j][current_solution[j][m]]
                if j > 0:
                    start_times[j][m] = max(stop_times[j-1][m], start_times[j-1][m]) + idle_times[j][m]
                else:
                    start_times[j][m] = 0
                stop_times[j][m] = start_times[j][m] + PROC[j][current_solution[j][m]]
                if m < N_MACHINES - 1:
                    idle_times[j][m+1] = max(0, stop_times[j][m] - start_times[j][m+1])
        
        temperature *= cooling_rate

    return best_solution, best_time, idle_times, start_times, stop_times


In [39]:

best_solution, best_time, idle_times, start_times, stop_times = simulated_annealing(max_iterations=1000, initial_temperature=1000, cooling_rate=0.95)
print("Best solution:", best_solution)
print("Best time:", best_time)


Best solution: [[1, 3, 2, 0], [1, 0, 3, 2], [1, 3, 2, 0], [3, 0, 1, 2], [0, 3, 2, 1], [2, 1, 0, 3]]
Best time: 27


In [40]:
def print_schedule(schedule, idle_times, start_times, stop_times):
    row = ""
    row += '|---------|'
    for m in range(N_MACHINES):
        row += f"------------|" \
                f"------------|"
    row += '\n'
    row += '|         |'
    for m in range(N_MACHINES):
        row += f"            |" \
                f" Machine: {m:1d} |"
    row += '\n'
    for j in range(N_JOBS):
        row += '|---------|'
        for m in range(N_MACHINES):
            row += f"------------|" \
                   f"------------|"
        row += '\n'
        row += '|         |'
        for m in range(N_MACHINES):
            row += f"       {' ':4s} | " \
                   f"Idle: {idle_times[j][m]:4d} |"  
        row += '\n'
        row += '|---------|'
        for m in range(N_MACHINES):
            row += f"------------|" \
                   f"------------|"
        row += '\n'
        row += '|         |'
        for m in range(N_MACHINES):
            row += f"       {' ':4s} | " \
                   f"Start:{start_times[j][m]:4d} |"
        row += '\n' 
        row += f'| Job: {j:2d} |'
        for m in range(N_MACHINES):
            row += f" Wait: {idle_times[j][m]:4d} | " \
                   f"Proc: {PROC[j][schedule[j][m]]:4d} |"
        row += '\n'
        row += '|         |'
        for m in range(N_MACHINES):
            row += f"       {' ':4s} | " \
                   f"Stop: {stop_times[j][m]:4d} |"  
        row += '\n' 
    row += '|---------|'
    for m in range(N_MACHINES):
        row += f"------------|" \
                f"------------|"
    row += '\n'
    print(row)



In [41]:
# Print the schedule
print_schedule(best_solution, idle_times, start_times, stop_times)


|---------|------------|------------|------------|------------|------------|------------|------------|------------|
|         |            | Machine: 0 |            | Machine: 1 |            | Machine: 2 |            | Machine: 3 |
|---------|------------|------------|------------|------------|------------|------------|------------|------------|
|         |            | Idle:    0 |            | Idle:    1 |            | Idle:    2 |            | Idle:    6 |
|---------|------------|------------|------------|------------|------------|------------|------------|------------|
|         |            | Start:   0 |            | Start:   0 |            | Start:   0 |            | Start:   0 |
| Job:  0 | Wait:    0 | Proc:    1 | Wait:    1 | Proc:    2 | Wait:    2 | Proc:    6 | Wait:    6 | Proc:    4 |
|         |            | Stop:    1 |            | Stop:    2 |            | Stop:    6 |            | Stop:    4 |
|---------|------------|------------|------------|------------|---------

In [42]:
 #Calculate Accuracy
accuracy = (best_time / int(pulp.value(prob.objective))) * 100

In [43]:
# Print Results
print("Makespan (Best time) using Simulated Annealing:", best_time)
print("Optimal Makespan from ILP Solution:", int(pulp.value(prob.objective)))
print("Accuracy of Simulated Annealing Solution:", accuracy, "%")

Makespan (Best time) using Simulated Annealing: 27
Optimal Makespan from ILP Solution: 40
Accuracy of Simulated Annealing Solution: 67.5 %


In [44]:

# Optimal solution ()
optimal_time = int(pulp.value(prob.objective))

# Greedy algorithm result
greedy_result, best_time = greedy_schedule(PROC)

# Simulated annealing result
annealing_result, annealing_time, annealing_idle_times, annealing_start_times, annealing_stop_times = simulated_annealing(max_iterations=1000, initial_temperature=1000, cooling_rate=0.95)

In [45]:
# Evaluation metrics
greedy_makespan = best_time
annealing_makespan = annealing_time

In [46]:
# Print the evaluation metrics
print("Evaluation Metrics:")
print(f"Optimal Makespan: {optimal_time}")
print(f"Greedy Makespan: {greedy_makespan}")
print(f"Simulated Annealing Makespan: {annealing_makespan}")

Evaluation Metrics:
Optimal Makespan: 40
Greedy Makespan: 30
Simulated Annealing Makespan: 27
